In [ ]:
import numpy as np
import sep

region = "10050+0270"
filter = "1" # 1, 2, 4

filename = "SMOG_data/" + region + "/SMOG_" + region + "_mosaic_I" + filter + ".fits"
# filename = "galaxies.fits"

In [ ]:
import subprocess

def run_sextractor(image_file, config_file='default.sex', params_file='default.param', conv_file='gauss_4.0_7x7.conv'):
    command = [
        'sex',  # SExtractor executable name
        image_file,  # Input image file
        '-c', config_file,  # Configuration file
        '-PARAMETERS_NAME', params_file,  # Parameters file
        '-FILTER_NAME', conv_file  # Convolution mask
    ]

    # Call the SExtractor command
    result = subprocess.run(command, capture_output=True, text=True)

    # Check if the command was successful
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
    else:
        print(f"Output: {result.stdout}")

# Run for the three images.
run_sextractor("SMOG_data/" + region + "/SMOG_" + region + "_mosaic_I1.fits")
run_sextractor("SMOG_data/" + region + "/SMOG_" + region + "_mosaic_I2.fits")
run_sextractor("SMOG_data/" + region + "/SMOG_" + region + "_mosaic_I4.fits")

In [ ]:
# Additional setup for reading the test image and displaying plots.
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams["figure.figsize"] = [10.0, 8.0]

In [ ]:
# Read image into standard 2D NumPy array.
file = fits.open(filename)
full = file[0].data

data = np.ascontiguousarray(full[5500:6800, 0:800]).byteswap().newbyteorder()

In [ ]:
# Show the image.
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation="nearest", cmap="gray", vmin=m-s, vmax=m+s, origin="lower")
plt.colorbar()

In [ ]:
# Measure a spatially varying background on the image.
bkg = sep.Background(data)

In [ ]:
# Get a "global" mean and noise of the image background.
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# Evaluate background as 2D array with same size as original image.
bkg_image = bkg.back()

In [ ]:
# Show the background.
plt.imshow(bkg_image, interpolation="nearest", cmap="gray", origin="lower")
plt.colorbar()

In [ ]:
# Evaluate the background noise as 2D array with same size as original image.
bkg_rms = bkg.rms()

In [ ]:
# Show the background noise.
plt.imshow(bkg_rms, interpolation="nearest", cmap="gray", origin="lower")
plt.colorbar()

In [ ]:
# Subtract the background
data_sub = data - bkg

In [ ]:
# Default SExtractor kernel.
# kernel = np.array([[1, 2, 1],
#                    [2, 4, 2],
#                    [1, 2, 1]])

# Gaussian 4.0 7x7 kernel.
kernel = np.array([[0.047454, 0.109799, 0.181612, 0.214776, 0.181612, 0.109799, 0.047454],
                   [0.109799, 0.254053, 0.420215, 0.496950, 0.420215, 0.254053, 0.109799],
                   [0.181612, 0.420215, 0.695055, 0.821978, 0.695055, 0.420215, 0.181612],
                   [0.214776, 0.496950, 0.821978, 0.972079, 0.821978, 0.496950, 0.214776],
                   [0.181612, 0.420215, 0.695055, 0.821978, 0.695055, 0.420215, 0.181612],
                   [0.109799, 0.254053, 0.420215, 0.496950, 0.420215, 0.254053, 0.109799],
                   [0.047454, 0.109799, 0.181612, 0.214776, 0.181612, 0.109799, 0.047454]])

# # Tophat 3.0 3x3 kernel.
# kernel = np.array([[0.560000, 0.980000, 0.560000],
#                    [0.980000, 1.000000, 0.980000],
#                    [0.560000, 0.980000, 0.560000]])

# Set parameters.
thresh = 5
error = bkg.globalrms
min_area = 2

objects = sep.extract(data_sub, thresh, err=error, filter_kernel=kernel, minarea=min_area)

In [ ]:
# How many objects were detected
len(objects)

In [ ]:
from matplotlib.patches import Ellipse

# Plot background-subtracted image.
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation="none", cmap="gray",
               vmin=m-s, vmax=m+s, origin="lower")

# Plot an ellipse for each object.
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects["theta"][i] * 180.0 / np.pi)
    e.set_facecolor("none")
    e.set_edgecolor("red")
    ax.add_artist(e)

In [ ]:
# Available fields.
objects.dtype.names

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# Show the first 10 objects' results.
# for i in range(10):
#     print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

In [ ]:
print(objects['x'])

In [ ]:
print(objects['y'])

In [ ]:
print(objects['a'])

In [ ]:
print(objects['b'])

In [ ]:
print(objects['theta'])